In [6]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
#from pystackreg import StackReg
#from plantcv import plantcv as pcv
#from VascGraph.Skeletonize import Skeleton

#from VascGraph.GraphIO import ReadStackMat
#from VascGraph.GraphLab import StackPlot
#from VascGraph.Tools.VisTools import visG
#from VascGraph.Tools.CalcTools import fixG
#from VascGraph.GraphIO import WritePajek

In [7]:
#!pip install pystackreg

In [8]:
def remove_small_comps_3d(image, thresh = 1000):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

In [9]:
directory = Path('james_preds')
files  = directory.glob('*_seg_warped.tif')
files = sorted([x.as_posix() for x in files])

In [5]:
files

['james_preds/XYZres006_0001_seg_warped.tif',
 'james_preds/XYZres006_seg_warped.tif',
 'james_preds/XYZres007_0001_seg_warped.tif',
 'james_preds/XYZres007_seg_warped.tif',
 'james_preds/XYZres008_0001_seg_warped.tif',
 'james_preds/XYZres008_seg_warped.tif',
 'james_preds/XYZres009_0001_seg_warped.tif',
 'james_preds/XYZres009_seg_warped.tif',
 'james_preds/XYZres010_0001_seg_warped.tif',
 'james_preds/XYZres010_seg_warped.tif',
 'james_preds/XYZres011_0001_seg_warped.tif',
 'james_preds/XYZres011_seg_warped.tif',
 'james_preds/XYZres012_0001_seg_warped.tif',
 'james_preds/XYZres012_seg_warped.tif',
 'james_preds/XYZres013_0001_seg_warped.tif',
 'james_preds/XYZres013_seg_warped.tif',
 'james_preds/XYZres014_0001_seg_warped.tif',
 'james_preds/XYZres014_seg_warped.tif',
 'james_preds/XYZres015_0001_seg_warped.tif',
 'james_preds/XYZres015_seg_warped.tif',
 'james_preds/XYZres016_0001_seg_warped.tif',
 'james_preds/XYZres016_seg_warped.tif',
 'james_preds/XYZres017_0001_seg_warped.tif

In [10]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    seg = io.imread(file)
    seg = seg.astype('int8')
    seg = (seg==1)*1
    seg = fill_holes(seg)
    #np.save(re.sub('mean','seg',file),seg)
    savemat(re.sub('_seg_warped.tif','_seg_warped.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(binary_dilation(binary_dilation(seg)))))})
    tif.imwrite(re.sub('_seg_warped.tif','_seg_warped_filled.tif',file),seg)

100%|██████████| 94/94 [34:47<00:00, 22.21s/it]


In [17]:
np.max(seg)

1.0

In [ ]:
files_seg_0001 = directory.glob('*_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
#files_seg_0001

In [64]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('seg_warped.tif','skel_warped.mat',file)
    seg = np.load(file)
    skel = loadmat(skel_file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    skel = remove_small_comps_3d(skel,10)
    dst_tsfm = distance_transform_edt(seg)*0.99436
    dst_tsfm[dst_tsfm==0]=0.00001
    skel_dst = skel*dst_tsfm

    
    np.save(re.sub('seg','dst_skel',file),skel_dst)
    io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
    graph = sknw.build_sknw(skel_dst, multi=False)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    nx.write_gpickle(graph,re.sub('_seg.npy','.pickle',file))
    nx.write_gpickle(graph_0001,re.sub('_seg.npy','.pickle',file_0001))
    #raph.to_csv(re.sub('seg.npy','graph.csv',file))

  0%|          | 0/47 [00:00<?, ?it/s]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres006_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres006_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
  2%|▏         | 1/47 [00:34<26:41, 34.81s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres007_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres007_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
  4%|▍         | 2/47 [01:11<26:55, 35.90s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres008_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.t

/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres026_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
 45%|████▍     | 21/47 [11:44<14:13, 32.82s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres027_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres027_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
 47%|████▋     | 22/47 [12:17<13:38, 32.72s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres028_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres028_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
 

 87%|████████▋ | 41/47 [22:55<03:19, 33.29s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres047_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres047_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
 89%|████████▉ | 42/47 [23:26<02:42, 32.58s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres048_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_239095/2239722327.py:29: UserWarning: james_preds/XYZres048_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
 91%|█████████▏| 43/47 [23:58<02:10, 32.61s/it]/tmp/ipykernel_239095/2239722327.py:24: UserWarning: james_preds/XYZres049_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy',

In [22]:
skel_dst_0001.shape

(507, 507, 252)

In [31]:
re.sub('_seg.npy','.pickle',file)

'james_preds/XYZres006_0001.pickle'

In [ ]:
skel_file = re.sub('seg.npy','skel.mat',file)

In [27]:
graph[0][39]

{'pts': array([[  3, 419,  33],
        [  4, 419,  33],
        [  5, 419,  33],
        [  6, 419,  32],
        [  7, 419,  31],
        [  8, 419,  31],
        [  9, 419,  31],
        [ 10, 419,  30],
        [ 11, 419,  29],
        [ 12, 419,  28],
        [ 13, 419,  27],
        [ 14, 419,  27],
        [ 15, 420,  26],
        [ 16, 420,  25],
        [ 17, 420,  25],
        [ 18, 420,  25],
        [ 19, 420,  25],
        [ 20, 420,  25],
        [ 21, 420,  24],
        [ 22, 420,  23],
        [ 23, 420,  22],
        [ 24, 420,  21],
        [ 25, 420,  20],
        [ 26, 420,  19],
        [ 26, 420,  18],
        [ 26, 420,  17],
        [ 27, 421,  16],
        [ 28, 421,  15],
        [ 29, 421,  14],
        [ 30, 421,  14],
        [ 31, 421,  13],
        [ 31, 421,  12],
        [ 32, 421,  11],
        [ 33, 421,  10],
        [ 34, 421,  10],
        [ 35, 421,   9],
        [ 36, 421,   9],
        [ 37, 421,   8],
        [ 38, 421,   9],
        [ 39, 421,

In [ ]:
re.sub('seg.npy','graph.csv',file_0001)

In [ ]:
files[22:24]